In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [42]:
pd.options.mode.chained_assignment = None

file_2012 = pd.read_sas('LLCP2012.XPT')

fip_state = pd.read_csv('fips_to_state.csv')

file_2012.head()

,_STATE,_GEOSTR,_DENSTR2,PRECALL,REPNUM,REPDEPTH,FMONTH,IDATE,IMONTH,IDAY,...,_PNEUMO2,_RFSEAT2,_RFSEAT3,_RFMAM2Y,_MAM502Y,_RFPAP32,_RFPSA21,_RFBLDS2,_RFSIGM2,_AIDTST3
0,1.0,7.0,1.0,1.0,20118.0,3.0,2.0,b'02162012',b'02',b'16',...,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,2.0,2.0
1,1.0,8.0,1.0,1.0,10119.0,17.0,1.0,b'01052012',b'01',b'05',...,NaN,1.0,1.0,NaN,NaN,NaN,1.0,2.0,1.0,1.0
2,1.0,8.0,2.0,1.0,10126.0,5.0,1.0,b'01032012',b'01',b'03',...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,9.0,1.0,2.0
3,1.0,8.0,1.0,1.0,10128.0,9.0,1.0,b'01192012',b'01',b'19',...,2.0,1.0,1.0,1.0,1.0,NaN,NaN,2.0,2.0,2.0
4,1.0,8.0,1.0,1.0,10130.0,26.0,1.0,b'01062012',b'01',b'06',...,NaN,1.0,1.0,1.0,1.0,NaN,NaN,2.0,1.0,2.0


In [43]:
# Re-naming the _STATE to fips for merging the FIPS DF and New DF# Re-nam 
file_2012 = file_2012.rename(columns={"_STATE":"fips"})
file_2012 = file_2012.merge(fip_state,on="fips")
# Dropping the fips column as it is not required further
file_2012.drop(["fips"],axis=1,inplace=True)

In [44]:
#create a new dataframe with the columns we want#create 
new_df = file_2012[['state_abbr', 'IDATE' , '_BMI5CAT', '_RFBMI5', '_AGE_G','SEX']]

new_df.head()

,state_abbr,IDATE,_BMI5CAT,_RFBMI5,_AGE_G,SEX
0,AL,b'02162012',3.0,2.0,5.0,1.0
1,AL,b'01052012',3.0,2.0,4.0,1.0
2,AL,b'01032012',4.0,2.0,6.0,2.0
3,AL,b'01192012',2.0,1.0,6.0,2.0
4,AL,b'01062012',4.0,2.0,5.0,2.0


In [48]:
#fill in columns with data based on the 2016 codebook
#reference: https://www.cdc.gov/brfss/annual_data/2016/pdf/codebook16_llcp.pdf
new_df["IDATE"] = new_df["IDATE"].str.decode("utf-8")
new_df["_BMI5CAT"] = new_df["_BMI5CAT"].replace({1:"Underweight", 2.0:"Normal Weight", 3.0:"Overweight", 4.0:"Obese", None:"Don't know/Refused/Missing"})
new_df["_RFBMI5"]= new_df["_RFBMI5"].replace({1.0:"No", 2.0:"Yes", 9.0:"Don’t know/Refused/Missing"})
new_df["_AGE_G"] = new_df["_AGE_G"].replace({1.0:"Age 18 to 24", 2.0:"Age 25 to 34", 3.0:"Age 35 to 44", 4.0:"Age 45 to 54", 5.0: "Age 55 to 64", 6.0: "Age 65 or older"})
new_df["SEX"] = new_df["SEX"].replace({1.0:"Male", 2.0:"Female", 9.0:"Refused"})

new_df = new_df.rename(columns={"state_abbr": "State", 
                              "IDATE": "Date", 
                              "_BMI5CAT": "BMI Category", 
                              "_RFBMI5": "BMI Over 25", 
                              "_AGE_G": "Age Group",
                              "SEX": "Sex"})

new_df.head()

KeyError: 'IDATE'

In [46]:
#save to csv
new_df.to_csv("ObesityAgeAndSex_2012.csv",index=False)